<a href="https://colab.research.google.com/github/innsvi/python_for_ds_task/blob/main/HW_15_4_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90_%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%96%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from statsmodels.stats.weightstats import ztest

In [2]:
import numpy as np
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# вхідні параметри
p1 = 0.19          # базове утримання 19%
p2 = 0.21          # зміни
alpha = 0.05       # рівень значущості
power = 0.80       # бажана сила тесту

# ефект (наскільки відрізняються пропорції) у стандартному вигляді
effect_size = proportion_effectsize(p1, p2)

# розрахунок потрібного розміру вибірки на ГРУПУ (A і B)
analysis = NormalIndPower()
n_per_group = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=1)

print("КРОК 1: Планування A/B тесту")
print(f"p1 (baseline) = {p1}")
print(f"p2 (target)   = {p2}")
print(f"alpha         = {alpha}")
print(f"power         = {power}")
print(f"effect_size   = {effect_size:.4f}")
print(f"n_per_group   = {np.ceil(n_per_group):.0f} користувачів у кожній групі")
print(f"total_n       = {np.ceil(n_per_group)*2:.0f} користувачів всього")


КРОК 1: Планування A/B тесту
p1 (baseline) = 0.19
p2 (target)   = 0.21
alpha         = 0.05
power         = 0.8
effect_size   = -0.0500
n_per_group   = 6276 користувачів у кожній групі
total_n       = 12552 користувачів всього


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/data/cookie_cats.csv')

In [5]:
# подивимось на перші рядки
df.head()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [6]:
# cереднє утримання на 7 день по версіях
retention_by_version = df.groupby("version")["retention_7"].mean()

retention_by_version


,retention_7
version,
gate_30,0.190201
gate_40,0.182000


**gate_30** має вище утримання на 7 день

**gate_40** має нижче утримання

Оскільки gate_30 виглядає кращою, ми перевіряємо, чи це не випадковість.

Нульова гіпотеза:
H0​: pgate_30​=pgate_40​ (Утримання на 7 день однакове для обох версій.)

Альтернативна гіпотеза:
Ha​: pgate_30​>pgate_40​ (Версія gate_30 має вище утримання на 7 день.)

Тип тесту: правосторонній z-test для двох пропорцій


3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [7]:
from statsmodels.stats.proportion import proportions_ztest

# кількість користувачів у кожній версії
n_gate_30 = df[df["version"] == "gate_30"].shape[0]
n_gate_40 = df[df["version"] == "gate_40"].shape[0]

# кількість "успіхів" (retention_7 == True)
success_gate_30 = df[df["version"] == "gate_30"]["retention_7"].sum()
success_gate_40 = df[df["version"] == "gate_40"]["retention_7"].sum()

print("gate_30: успіхи =", success_gate_30, ", n =", n_gate_30)
print("gate_40: успіхи =", success_gate_40, ", n =", n_gate_40)

gate_30: успіхи = 8502 , n = 44700
gate_40: успіхи = 8279 , n = 45489


In [8]:
# z-test для двох пропорцій
z_stat, p_value = proportions_ztest(
    count=[success_gate_30, success_gate_40],
    nobs=[n_gate_30, n_gate_40],
    alternative='larger'   # gate_30 > gate_40
)

print("Z-TEST ДЛЯ ДВОХ ПРОПОРЦІЙ (retention_7)")
print(f"z-статистика = {z_stat:.4f}")
print(f"p-value      = {p_value:.6f}")


Z-TEST ДЛЯ ДВОХ ПРОПОРЦІЙ (retention_7)
z-статистика = 3.1644
p-value      = 0.000777


За результатами A/B тестування виявлено статистично значущу різницю в утриманні користувачів на 7-й день між версіями гри gate_30 та gate_40. Середнє значення показника retention_7 для версії gate_30 становить приблизно 19.02%, тоді як для версії gate_40 — близько 18.20%.

Проведений правосторонній z-тест для двох пропорцій показав значення p-value = 0.000777, що є значно меншим за рівень значущості α = 0.05. У зв’язку з цим нульову гіпотезу про відсутність різниці між версіями було відхилено.

Отримані дані надають достатні статистичні підстави стверджувати, що версія gate_30 забезпечує вищий рівень утримання користувачів на 7-й день після встановлення гри порівняно з версією gate_40. З практичної точки зору, це означає, що версію gate_30 доцільно обрати як основну.

Довірчі інтервали для показника retention_7 у версіях gate_30 та gate_40 не перетинаються (або мають мінімальний перетин). Це означає, що істинні значення утримання користувачів для різних версій гри з високою ймовірністю відрізняються.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [9]:
from scipy.stats import chi2_contingency

In [11]:
alpha = 0.05

# контингентна таблиця (частоти)
ct = pd.crosstab(df["version"], df["retention_7"])
print("Контингентна таблиця (version x retention_7):")
print(ct)

# сhi-square test of independence
chi2, p_value, dof, expected = chi2_contingency(ct)


print("CHI-SQUARE TEST (незалежність)")
print(f"chi2 = {chi2:.4f}")
print(f"dof  = {dof}")
print(f"p-value = {p_value:.6f}")
print(f"alpha   = {alpha}")

# рішення
print("\nВисновок:")
if p_value < alpha:
    print("✅ Відхиляємо H0: є статистично значуща залежність між версією гри та retention_7.")
else:
    print("❌ Не відхиляємо H0: недостатньо доказів залежності між версією гри та retention_7.")

# Очікувані частоти переконатися, що тест коректний
expected_df = pd.DataFrame(expected, index=ct.index, columns=ct.columns)
print("\nОчікувані частоти (якби H0 була істинна):")
print(expected_df.round(2))


Контингентна таблиця (version x retention_7):
retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279
CHI-SQUARE TEST (незалежність)
chi2 = 9.9591
dof  = 1
p-value = 0.001601
alpha   = 0.05

Висновок:
✅ Відхиляємо H0: є статистично значуща залежність між версією гри та retention_7.

Очікувані частоти (якби H0 була істинна):
retention_7    False   True 
version                     
gate_30      36382.9  8317.1
gate_40      37025.1  8463.9


За результатами x-тесту незалежності на рівні значущості 5% бачимо статистично значущу залежність між версією гри та утриманням користувачів на 7-й день після реєстрації (p-value = 0.001601 < 0.05). Отже, нульову гіпотезу про незалежність змінних відхиляємо. Це свідчить про те, що версія гри впливає на ймовірність утримання користувачів на 7-й день.